In [ ]:
#Importing and Installing the necessary packages
import selenium 
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
from tokenize import String
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException
import re
import json
import csv
import pandas as pd

In [ ]:
#defining the dataframe
df = pd.DataFrame(columns=['Team Name','Player Name','Twitter ID','Instagram ID','Player Position','Shoots','Born','Country','NBA Debut','Experience','Summary','G','PTS','TRB','AST','FG%','FG3%','FT%','eFG%','PER','WS','Summary','G','PTS','TRB','AST','FG%','FG3%','FT%','eFG%','PER','WS'])

In [ ]:
df

In [ ]:
driver = webdriver.Chrome() 

driver.get("https://www.basketball-reference.com/")

x = driver.find_element(By.ID,'choose_player_team')
drop1 = Select(x)
count_options_team = str(len(drop1.options))
print(count_options_team)
count = 0

#loop to iterate through all the teams
# for team in range(1,int(count_options_team)):
for team in range(1,31):
    
    driver.get("https://www.basketball-reference.com/")
    time.sleep(3)

    if(team == 31):
            continue
    else:
        x = driver.find_element(By.ID,'choose_player_team')
        drop1 = Select(x)
        drop1.select_by_index(team)
        time.sleep(2)
        option = []
        o = drop1.first_selected_option
        option.append(o.text)
    
    y = driver.find_element(By.ID,'choose_player');
    drop2 = Select(y)
    count_options_player = str(len(drop2.options))
    print(count_options_player)

    #loop to iterate through all the players in every team
    for player in range(1,int(count_options_player)):

        count = count + 1 

        print(team)
        print(player)

        #choosing the ID tag and selecting the player based on index
        a = driver.find_element(By.ID,'choose_player');
        drop2 = Select(a)  
        drop2.select_by_index(player)
        time.sleep(2) 

        #Click on "Go" button to view the player details
        z = driver.find_element(By.XPATH, "//input[@name='go_button']") 
        z.click() 
        time.sleep(4) 

        #Now we are in the player page
        #try catch block to check if there is a more details button or not
        try:
            more_details = driver.find_element(By.XPATH, "//button[@id='meta_more_button']") 
            more_details.click() 
        except NoSuchElementException: 
            pass 
        
        #try catch block to get the "name" and the "number of p tags" 
        global detail 
        detail = [] 
        detail = option + detail
        try:
            name = driver.find_element(By.XPATH, "//div[@id='meta']//div[@class='nothumb']//h1//span")
            names = name.text
            detail.append(names)
            count_of_p = driver.find_elements(By.XPATH, "//div[@id='meta']//div[@class='nothumb']//p")
            number_of_p = len(count_of_p) 
        except:
            name = driver.find_element(By.XPATH, "//div[@id='meta']//div[2]/h1/span")
            names = name.text
            detail.append(names)
            count_of_p = driver.find_elements(By.XPATH, "//div[@id='meta']//div[2]/p")
            number_of_p = len(count_of_p)
        
        time.sleep(3)
        raw_details = []

        for x in range(1,number_of_p+1):
            try:
                details = driver.find_element(By.XPATH, f"//div[@id='meta']//div[@class='nothumb']//p[{x}]");
                x = details.text
                raw_details.append(x)
            except:
                details = driver.find_element(By.XPATH, f"//div[@id='meta']//div[2]/p[{x}]");
                x = details.text
                raw_details.append(x)
        print(raw_details)

        for i in range(0,2):
            x = raw_details[i].split(':')
            if x[0] != 'Pronunciation':
                y = raw_details[i].split('▪')
                break

        #Twitter and Instagram ID
        print(y)
        social_media = [' Twitter',' Instagram']
        if (len(y) == 2):
            q = y[1].split(':')
            print(q)
            if(q[0] == social_media[0]):
                detail.append(q[1])
                detail.append("N/A")
            else:
                detail.append("N/A")
                detail.append(q[1])
        else:           
            for i in range(1,len(y)):
                p = y[i].split(':')
                if p[0] == ' Twitter':
                    detail.append(p[1])
                else:
                    detail.append(p[1])
                    break
            else:
                detail.append("N/A")
                detail.append("N/A")

        #Player Position and Shoots
        for i in range(0,5):
            x = raw_details[i].split(':')
            if 'Position' in x:
                y = raw_details[i].split('▪')
                position = y[0].split(':')
                detail.append(position[1])
                shoots = y[1].split(':')
                detail.append(shoots[1])
                break
        else:
            detail.append("N/A")

        #Player Birth Details and Home Country
        for i in range(0,len(raw_details)):
            x = raw_details[i].split('(')[0].split(':')
            if 'Born' in x:
                detail.append(x[1])
                y = raw_details[i].split('in',1)[1]
                detail.append(y)
                break
        else:
            detail.append("N/A")
        
        #Player NBA Debut 
        for i in range(0,len(raw_details)):
            x = raw_details[i].split(':')
            if 'NBA Debut' in x:
                    detail.append(x[1])
                    break
        else:
            detail.append("N/A")

        #Player Experience
        for i in range(0,len(raw_details)):
            x = raw_details[i].split(':')
            if 'Experience' in x:
                detail.append(x[1])
                break
        else:
            detail.append("N/A")

        try:
            summary1 = driver.find_element(By.XPATH,"//div[@class='stats_pullout']//div//div//p")
            detail.append(summary1.text)

            for i in range(1,5):
                statistics1 = driver.find_element(By.XPATH,f"//div[@class='stats_pullout']//div[@class='p1']/div[{i}]/p")
                detail.append(statistics1.text)

            for i in range(1,5):
                statistics2 = driver.find_element(By.XPATH,f"//div[@class='stats_pullout']//div[@class='p2']/div[{i}]/p")
                detail.append(statistics2.text)

            for i in range(1,3):
                statistics3 = driver.find_element(By.XPATH, f"//div[@class='stats_pullout']//div[@class='p3']/div[{i}]/p")
                detail.append(statistics3.text)
        except:
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")

        try:
            summary2 = driver.find_element(By.XPATH,"//div[@class='stats_pullout']//div//div//p[2]")
            detail.append(summary2.text)

            for i in range(1,5):
                statistics11 = driver.find_element(By.XPATH,f"//div[@class='stats_pullout']//div[@class='p1']/div[{i}]/p[2]")
                detail.append(statistics11.text)

            for i in range(1,5):
                statistics12 = driver.find_element(By.XPATH,f"//div[@class='stats_pullout']//div[@class='p2']/div[{i}]/p[2]")
                detail.append(statistics12.text)

            for i in range(1,3):
                statistics13 = driver.find_element(By.XPATH, f"//div[@class='stats_pullout']//div[@class='p3']/div[{i}]/p[2]")
                detail.append(statistics13.text)
        except:
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")
            detail.append("N/A")

        print(detail)
        print(team)
        df.loc[count] = [detail[0],detail[1],detail[2],detail[3],detail[4],detail[5],detail[6],detail[7],detail[8],detail[9],detail[10],detail[11],
                        detail[12],detail[13],detail[14],detail[15],detail[16],detail[17],detail[18],detail[19],detail[20],detail[21],detail[22],
                        detail[23],detail[24],detail[25],detail[26],detail[27],detail[28],detail[29],detail[30],detail[31]]

        driver.get("https://www.basketball-reference.com/")
        time.sleep(3)

        x = driver.find_element(By.ID,'choose_player_team')
        drop1 = Select(x)
        drop1.select_by_index(team)
        time.sleep(2)

In [ ]:
df

In [ ]:
df.to_csv('Data4.csv')